In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

import pandas as pd, numpy as np, tensorflow as tf

In [3]:
def mean_absolute_percentage_error(y_true, y_pred):
    return tf.reduce_mean(tf.abs((y_true-y_pred) / y_true)) * 100.0

In [4]:
#df = pd.read_feather('E:/Skóli/HÍ/Vélaverkfræði Master HÍ/Lokaverkefni/Data/merged-full-25ms-24hr-28-2-24.feather')
#df = pd.read_feather('E:/Skóli/HÍ/Vélaverkfræði Master HÍ/Lokaverkefni/Data/merged-w-landscape-full-25ms-24hr-28-2-24.feather')
df = pd.read_feather('E:/Skóli/HÍ/Vélaverkfræði Master HÍ/Lokaverkefni/Data/merged-full-W-Landscape-And-Station-Elevations-25ms-24hr-11-3-24.feather')
df = df[df.f < df.fg]
df['gust_factor'] = df.fg / df.f

df = df.drop(['f', 'fg', 'fsdev', 'd', 'dsdev', 'longitude', 'latitude', 'X', 'Y', 'time', 'stod'], axis = 1)# + [f'Landscape_{i}' for i in range(70)], axis = 1)

df_unfolded = df.elevations.apply(pd.Series)
df = pd.concat([df, df_unfolded], axis = 1)

df = df.dropna()
df = df.reset_index(drop = True)

In [5]:
df.iloc[:, -70:] = df.iloc[:, -70:].sub(df.station_elevation, axis = 0)

In [6]:
from sklearn.decomposition import PCA

df_landscape_elevation = df.iloc[:, -70:]

df_landscape_elevation = (df_landscape_elevation - df_landscape_elevation.mean()) / df_landscape_elevation.std()

n_components = 5

pca = PCA(n_components=n_components)
compressed_features = pca.fit_transform(df_landscape_elevation)

compressed_df = pd.DataFrame(data = compressed_features, columns = ['PC' + str(i) for i in range(n_components)])

df  = pd.concat([df, compressed_df], axis = 1)

In [7]:
y = df.gust_factor
X = df.drop(['gust_factor'], axis = 1)
X = X.drop(['ws_15', 'ws_250', 'ws_500', 'wd_15', 'wd_250', 'wd_500', 'p_15', 'p_250', 'p_500', 't_15', 't_250', 't_500', 'N_01', 'N_12', 'landscape_points', 'elevations']
           + [i for i in range(70)], axis = 1)

# Changing the type of X,y so as to work with Tensorflow
X, y = X.values.astype(np.float32), y.values.astype(np.float32)
scaler = StandardScaler()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)
X_train = scaler.fit_transform(X_train)
X_val = scaler.fit_transform(X_val)
X_test = scaler.fit_transform(X_test)

In [8]:
model = Sequential()
model.add(LSTM(25, input_shape=(X_train.shape[1], 1), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(200, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(25))
model.add(Dense(1))  # Output layer for regression


c:\Users\Brynjar Geir\Documents\lokaverkefni_vel\lokaverkefni_vel\code\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [9]:
model.compile(optimizer='adam', loss = mean_absolute_percentage_error)
model.fit(X_train, y_train, epochs = 100, batch_size = 512, validation_data=(X_val, y_val))

Epoch 1/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 18s 283ms/step - loss: 77.1322 - val_loss: 11.1654
Epoch 2/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 200ms/step - loss: 10.7998 - val_loss: 8.9034
Epoch 3/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 204ms/step - loss: 8.6951 - val_loss: 8.3824
Epoch 4/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 5s 237ms/step - loss: 8.6292 - val_loss: 8.3265
Epoch 5/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 201ms/step - loss: 8.5329 - val_loss: 8.3142
Epoch 6/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 179ms/step - loss: 8.1468 - val_loss: 8.2219
Epoch 7/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 196ms/step - loss: 8.1716 - val_loss: 8.0480
Epoch 8/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 167ms/step - loss: 8.1130 - val_loss: 8.1426
Epoch 9/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 170ms/step - loss: 8.0441 - val_loss: 7.9305
Epoch 10/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 172ms/step - loss: 7.7184 - val_loss: 8.0479
Epoch 11/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 167ms/step - loss: 7.9625 - val_loss: 7.7537
Epoch 12/100
22/22 ━━━━━━━━━━━━━━━━━━

In [10]:
y_pred = model.predict(X_test)
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f"MAPE: {mape} %")

115/115 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step
MAPE: 8.512425422668457 %


In [65]:
model.save("./saved_models/rnn11-3-24.keras")